In [2]:
import math
import re
import sqlite3
import time
import pandas as pd
pd.set_option('display.max_columns', None)
#import plotly.graph_objects as go
#import plotly.express as px
from bs4 import BeautifulSoup as bs4
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By

from sqlalchemy import MetaData, text, Column, Integer, String, ForeignKey, Table, create_engine, Float, Boolean, DateTime
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

meta = MetaData()
engine = create_engine('sqlite:///fantasy_data.db', echo=False)

#from optimize_lineup import Optimized_Lineups
from fantasy_utils import Fantasy_Projections as fp
import fantasy_utils as fu
fp = fu.Fantasy_Projections()

In [6]:
conn = sqlite3.connect('fantasy_data.db')
conn.create_function('sqrt', 1, math.sqrt)

In [ ]:
def login():
    driver = webdriver.Chrome("C:\\ProgramData\\Anaconda3\\WebDriver\\bin\\chromedriver.exe")
    driver.get('https://www.cbssports.com/login?master_product=150&xurl=https%3A%2F%2Fwww.cbssports.com%2Flogin')
    search_box = driver.find_element_by_id('userid')
    search_box.send_keys('gostros09')
    search_box = driver.find_element_by_id('password')
    search_box.send_keys('Segneri9A')
    search_box.submit()
    return driver

In [ ]:
def scrape_cbs_owner_weekly_roster(soup, ids, owner, wk):
    id_dict = {i.text:int(i['href'].split('/')[-1]) for i in soup.find_all("a", {"class": "playerLink"}, href=True) if i.text!=''}
    df = pd.DataFrame()
    for idx in range(2):
        t = pd.read_html(str(soup.find_all('table')[1:25]))[idx]
        t.columns = t.columns.droplevel().droplevel()
        t = t[['Pos', 'Players', 'Salary']]
        t['Week'] = wk
        t['Owner'] = owner
        
        if idx==0: # for batters
            reserves_idx = t[t['Pos']=='Reserves'].index.values[0]
            t = pd.concat([t.iloc[:reserves_idx], t.iloc[reserves_idx+1:]])
            t.loc[:reserves_idx, 'Decision'] = 'Start'
            t.loc[reserves_idx:, 'Decision'] = 'Sit'
        else: # for pitchers
            reserves_idx = t[t['Pos']=='Reserves'].index.values[0]
            t = pd.concat([t.iloc[:reserves_idx], t.iloc[reserves_idx+1:-1]])
            t.loc[:reserves_idx, 'Decision'] = 'Start'
            t.loc[reserves_idx:t.index.max(), 'Decision'] = 'Sit'
        
        try:
            t['Team'] = t.Players.apply(lambda x: x.split()[-1])
        except:
            pass
        try:
            t['Player'] = t.Players.apply(lambda x: ' '.join(x.split()[:-3]))
        except:
            pass

        t = t.merge(pd.Series(id_dict).to_frame('CBSID'), left_on='Player', right_index=True, how='left')
        t = t.merge(ids[['CBSID', 'IDFANGRAPHS', 'MLBID']], on='CBSID', how='left', indicator=True)[['Player', 'Owner', 'Week', 'Team', 'Pos', 'Decision', 'CBSID', 'MLBID', 'IDFANGRAPHS']]
        df = pd.concat([df,t])
    
    df.rename(columns={'Player':'CBSNAME'},inplace=True)
    return df

In [ ]:
def click_week(wk):
    details = driver.find_element_by_xpath("//div[@class ='select_form_div darkFilter']")
    res = details.find_elements_by_xpath("//li")
    for i in res:
        if i.text==wk:
            i.click()
    return

In [ ]:
def get_weekly_sit_or_start_by_owner(owner_name, owner_id, ids):
    driver.get(f"https://xdl.baseball.cbssports.com/teams/{owner_id}")
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, 0);")

    data = pd.DataFrame()
    for wk in range(1,28):
        if wk==1:
            driver.find_element_by_xpath(f"//div[@class ='select_form_div darkFilter']/div/span[text()='Off Season']").click()
            driver.implicitly_wait(4)
            driver.find_element_by_xpath(f"//div[@class ='select_form_div darkFilter']/ul/li[@value='/teams/roster-report/{owner_id}/1/']").click()
        else:
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(.5)
            driver.find_element_by_xpath(f"//div[@class ='select_form_div darkFilter']/div/span[text()='{wk-1}']").click()
            time.sleep(2)
            elem = driver.find_elements_by_xpath(f"//div[@class ='select_form_div darkFilter']/ul/li")
            driver.implicitly_wait(4)
            time.sleep(2)
            for e in elem:
                if e.text==str(wk):
                    e.click()

        time.sleep(2)
        html = driver.page_source
        soup = bs4(html, 'html.parser')
        data = pd.concat([data, scrape_cbs_owner_weekly_roster(soup, ids, owner_name, wk)])

    data.to_excel(f'data//{datetime.now().year}-{owner_name}-sit-start.xlsx', engine='openpyxl', index=False)
    return data

In [3]:
pd.read_sql('owners', engine).to_dict(orient='records')

[{'owner_id': 30, 'owner': '9 Grand Kids'},
 {'owner_id': 3, 'owner': 'Brewbirds'},
 {'owner_id': 36, 'owner': 'Charmer'},
 {'owner_id': 41, 'owner': 'Dirty Birds'},
 {'owner_id': 4, 'owner': 'Harveys Wallbangers'},
 {'owner_id': 27, 'owner': 'Lil Trump & the Ivanabees'},
 {'owner_id': 38, 'owner': 'Lima Time!'},
 {'owner_id': 44, 'owner': 'Roid Ragers'},
 {'owner_id': 1, 'owner': 'Trouble with the Curve'},
 {'owner_id': 29, 'owner': 'Ugly Spuds'},
 {'owner_id': 42, 'owner': 'Wiscompton Wu-Tang'},
 {'owner_id': 45, 'owner': 'Young Guns'}]

In [ ]:
owner_dict = {"9 Grand Kids":'30', 'Brewbirds':'3', 'Charmer':'36', 'Dirty Birds':'41', "Harveys Wallbangers":'4', 'Lil Trump & the Ivanabees':'27', 'Lima Time!':'38', 
                        'Roid Ragers':'44', 'Trouble with the Curve':'1', 'Ugly Spuds':'29', 'Wiscompton Wu-Tang':'42', 'Young Guns':'45'}
position_priority = ['C', '2B', '3B', 'SS', 'OF', '1B', 'MI', 'CI', 'DH', 'SP', 'RP']

In [ ]:
ids = fp.load_id_map()

In [ ]:
name_change = {"9 Gran...":'9 Grand Kids', 'Brewbi...':'Brewbirds', 'Charme...':'Charmer', 'Dirty ...':'Dirty Birds', 'Harvey...':'Harveys Wallgangers', 'Lil Tr...':'Lil Trump & the Ivanabees', 
               'Lima T...':'Lima Time!', 'Roid R...':'Roid Ragers', 'Troubl...':'Trouble with the Curve', 'Ugly S...':'Ugly Spuds', 'Wiscom...':'Wiscompton Wu-Tang', 'Young ...':'Young Guns'}

In [ ]:
team_list = ['Lima Time!', 'Ugly Spuds', 'Harveys Wallbangers', 'Roid Ragers', 'Charmer', 'Wiscompton Wu-Tang', '9 Grand Kids', 'Brewbirds', 'Trouble With The Curve', 'Lil Trump & The Ivanabees', 'Dirty Birds', 'Young Guns'] 
stat_list = ['BA', 'HR', 'R', 'RBI', 'SB', 'ERA', 'Sv+Hld', 'SO', 'W', 'WHIP']

In [ ]:
driver = login()

In [ ]:
driver.implicitly_wait(10)
driver.get('https://xdl.baseball.cbssports.com/stats/stats-main')
time.sleep(4)
driver.find_element_by_xpath(f"//a[@href ='/SP:RP']").click()
time.sleep(1.5)
driver.find_element_by_xpath(f"//a[@href ='/all']").click()
time.sleep(1.5)
driver.execute_script("window.scrollTo(0, 150);")
driver.find_element_by_xpath(f"//div[@class ='select_div_label_container selected_arrow']").click()
time.sleep(2)
elem = driver.find_elements_by_xpath(f"//ul[@class='form_select_ul combinedFilterUl3Cols']/li")


### Begin loop code

In [ ]:
# pitching
#driver.get(f'https://xdl.baseball.cbssports.com/stats/stats-main/all:SP:RP/period-{1}:p/standard/stats?print_rows=999')
# hitting
driver.get(f'https://xdl.baseball.cbssports.com/stats/stats-main/all:C:1B:2B:3B:SS:MI:CI:OF:DH/period-1:p/standard/stats?print_rows=9999')
# Then need to click "All"

In [ ]:
wk = 0
df = pd.DataFrame()

In [ ]:
wk

In [ ]:
wk += 1
html = driver.page_source
soup = bs4(html, 'html.parser')
data = pd.read_html(str(soup.find_all('table')[1:25]))[0]
data.columns = data.columns.droplevel()
data = data.iloc[:-1, 1:]
data['Rank'] = data['Rank'].astype(int)
data = data[(data['Avail']!='FA') | (data['Rank']<9999)]
data['Week'] = wk

try:
    data['Team'] = data.Player.apply(lambda x: x.split()[-1])
except:
    pass
try:
    data['Player'] = data.Player.apply(lambda x: ' '.join(x.split()[:-3]))
except:
    pass

id_dict = {int(i['href'].split('/')[-1]):[i.text, i['aria-label'].split(' ')[-1]] for i in soup.find_all("a", {"class": "playerLink"}, href=True) if i.text!=''}
id_df = pd.DataFrame(id_dict, index=['CBSNAME', 'Team']).T
data = data.merge(id_df.reset_index(), left_on=['Player', 'Team'], right_on=['CBSNAME', 'Team'], how='left')

df = pd.concat([df,data])
time.sleep(2)
if wk<27:
    driver.get(f'https://xdl.baseball.cbssports.com/stats/stats-main/all:C:1B:2B:3B:SS:MI:CI:OF:DH/period-{wk+1}:p/standard/stats?print_rows=9999')

In [ ]:
df.shape

In [ ]:
data

### End looping code

In [ ]:
df.rename(columns={'index':'cbsid'},inplace=True)
df = df.replace(name_change)
df.loc[df['Player']=='Logan O\'Hoppe', 'cbsid'] = 3166980
df.loc[df['Player']=='Travis d\'Arnaud', 'cbsid'] = 1730742
df.loc[df['Player']=='Brian O\'Keefe', 'cbsid'] = 3089588
df.loc[df['Player']=='Ke\'Bryan Hayes', 'cbsid'] = 2211187
df.loc[df['Player']=='Tyler O\'Neill', 'cbsid'] = 2171342
df.loc[df['Player']=='Ryan O\'Hearn', 'cbsid'] = 2211177
#df = df.merge(ids[['CBSID', 'IDFANGRAPHS', 'MLBID']], left_on='cbsid', right_on='CBSID', how='left', indicator=True)

In [ ]:
df = pd.read_excel('data//2023-weekly-stats-h.xlsx')

In [ ]:
df[df['cbsid'].isna()].Player.unique()

In [ ]:
#df.to_excel('data//2023-weekly-holds.xlsx')
#dfp = df.copy()

In [ ]:
dfh = df.copy()
#dfh.to_excel('data//2023-weekly-stats-new.xlsx')

In [ ]:
dfh.shape

In [ ]:
dfh = pd.read_excel('data//2023-weekly-stats-new.xlsx')
dfh.fillna(0,inplace=True)

In [ ]:
dfh['year'] = 2023
dfh.rename(columns={'Week':'week'},inplace=True)

In [ ]:
owners = pd.read_sql('roster', engine)
owners.shape

In [ ]:
owners.info()

In [ ]:
dfh.info()

In [ ]:
pd.pivot_table(owners, values='cbsid', aggfunc='count', index='owner_id', columns='week')#.sum().sum()

In [ ]:
pd.pivot_table(dfh, columns='week', values='cbsid', aggfunc='count')

In [ ]:
dfh[['cbsid', 'year', 'week', 'R', 'RBI', 'HR', 'SB', 'H', 'AB']].merge(owners.query('pos!="B" & pos!="SP"'), on=['cbsid', 'year', 'week'], how='inner').query('owner_id==36 & week==1')

In [ ]:
pd.pivot_table(dfh[['cbsid', 'year', 'week', 'R', 'RBI', 'HR', 'SB', 'H', 'AB']].merge(owners, on=['cbsid', 'year', 'week'], how='inner').query('pos!="B" & pos!="SP" & pos!="RP" & pos!="P"'),
               index='owner_id', columns='week', values='HR', aggfunc='sum')

In [ ]:
p = pd.read_excel('data//2023-weekly-stats-p.xlsx')
#p['year'] = 2023
#p.rename(columns={'Outs':'outs', 'Week':'week'},inplace=True)
#p.rename(columns={'H':'Ha', 'BB':'BBa', 'K':'SO'},inplace=True)

In [ ]:
p[p['cbsid'].isna()]

In [ ]:
#p['outs'] = p.INNs.apply(lambda x: (int(str(x).split('.')[0])*3) + int(str(x).split('.')[1]))
#p['IP'] = round(p['outs']/3.,2)
#p['ER'] = round(p['ERA']*p['IP']/9.,1)
#p['SvHld'] = p['S']+p['HD']

In [ ]:
p.to_excel('data//2023-weekly-stats-p.xlsx')

In [ ]:
pd.pivot_table(p, columns='week', values='cbsid', aggfunc='count').reset_index().sort_values(1, ascending=False)

In [ ]:
pd.pivot_table(p[['cbsid', 'year', 'week', 'W', 'SO', 'SvHld', 'ER', 'Ha', 'BBa', 'IP', 'outs']].merge(owners, on=['cbsid', 'year', 'week'], how='inner').query('pos=="SP" | pos=="RP" | pos=="P"'),
               index='owner_id', columns='week', values='SO', aggfunc='sum').sort_values(1,ascending=False)

In [ ]:
p[['cbsid', 'year', 'week', 'W', 'SO', 'SvHld', 'ER', 'Ha', 'BBa', 'IP', 'outs']].merge(owners, on=['cbsid', 'year', 'week'], how='inner').query('cbsid==2901324')

In [ ]:
dfh[['cbsid', 'year', 'week', 'R', 'RBI', 'HR', 'SB', 'H', 'AB']].merge(owners, on=['cbsid', 'year', 'week'], how='inner').query('cbsid==2901324')

In [ ]:
#dfh = dfh[['cbsid', 'year', 'week', 'R', 'RBI', 'HR', 'SB', 'H', 'AB']].sort_values(['cbsid', 'year', 'week'])
p = p[['cbsid', 'year', 'week',  'W', 'SO', 'SvHld', 'ER', 'Ha', 'BBa', 'IP', 'outs']].sort_values(['cbsid', 'year', 'week'])

In [ ]:
stats = dfh.merge(p, on=['cbsid', 'year', 'week'],how='outer')#.fillna(0)

In [ ]:
#for col in ['cbsid', 'R', 'RBI', 'HR', 'SB', 'H', 'AB', 'W', 'SO', 'SV+HLD', 'ER', 'Ha', 'BBa', 'outs']:
#    stats[col] = stats[col].astype(int)

In [ ]:
# Uncomment to append to stats table
# ! make sure not to fill nan values !
#stats.sort_values(['cbsid', 'year', 'week']).to_sql('stats', if_exists='replace', con=engine, index=False)

#### need a way to remove data where the player did not hit or did not pitch, or need to make two separate stats tables

In [ ]:
pd.pivot_table(stats, index='week', values=['R', 'RBI', 'HR', 'SB'], aggfunc=['mean', 'std'])

In [116]:
pd.read_sql("SELECT name FROM sqlite_master", engine)

,name
0,hitting
1,ix_hitting_index
2,hitters
3,sqlite_autoindex_hitters_1
4,players2022
5,players
6,ix_players_index
7,owners
8,drafted
9,eligibility
